In [2]:
import argparse
import sys
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torch.optim as optim
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
import torch.nn.functional as F
import tensorflow as tf

In [3]:
# import np vectors used in classification
avis = []
lab = []

labels = [["sinker",0], ["curve", 1]]
labels2 = [["ball", 1], ["strike", 0]]

split = int(len(avis)*3/4)

x_train, x_test, y_train, y_test = map(
    torch.tensor, (avis[:split], avis[split:], lab[:split], lab[split:]))

y_names = labels[:][0]
labels = labels[:][1]

In [23]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=2, padding=2)
        self.bn2 = nn.BatchNorm2d(32)


        self.conv5 = nn.Conv2d(32, 16, kernel_size=3, stride=2, padding=2)
        self.bn5 = nn.BatchNorm2d(16)
        self.conv6 = nn.Conv2d(16, 4, kernel_size=3, stride=2, padding=2)
        self.bn6 = nn.BatchNorm2d(4)

        self.fc1 = nn.Linear(3772, 128)
        self.fc2 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, 2)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)
        x = F.dropout(x, 0.3, training=self.training)

        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = F.max_pool2d(x, 2)
        x = F.dropout(x, 0.3, training=self.training)

        x = x.view(x.size(0), -1)  # Flatten layer
        x = F.relu(self.fc1(x))
        x = F.dropout(x, 0.3, training=self.training)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [12]:
import cv2
import numpy as np

def extract_frames(video_path, frame_indices):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if the video file is opened successfully
    if not cap.isOpened():
        print("Error: Unable to open video file.")
        return

    # Get the total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    rev_idxs = (total_frames-1)*np.ones(len(frame_indices))-np.array(frame_indices)
    # Extract frames at specified indices
    for idx in rev_idxs:
        # Set the frame position
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)

        # Read the frame
        ret, frame = cap.read()

        # Check if the frame is read successfully
        if ret:
            # Save the frame
            # cv2.imwrite(f"{output_folder}/frame_{idx}.jpg", frame)
            # print(f"Frame {idx} saved successfully.")
            break


        else:
            print(f"Error: Unable to read frame {idx}.")

    # Release the video capture object
    cap.release()
    return frame

# Example usage
video_path = "test.avi"
output_folder = "output_frames"
frame_indices = [0, 1, 2]  # Specify the indices of frames you want to extract

# Call the function to extract frames
frames = extract_frames(video_path, frame_indices)
print(frames[0].shape)
# print(frames[0])


(1280, 3)


In [13]:
def train(x_train, x_test, y_train, y_test):

    x_test = x_test.view(-1, 3, 720, 1280)
    x_train = x_train.view(-1, 3, 720, 1280)

    y_train = y_train.long()
    y_test = y_test.long()

    model = Network()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=.2)

    # create the train dataset and loader (following pytorch documentation)
    train_dataset = TensorDataset(x_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)

    epochs = 12
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            inputs = inputs.view(-1, 3, 720, 1280)
            labels = labels.view(-1)
            # print("labels: ", labels)
            optimizer.zero_grad()

            outputs = model(inputs.float())
            #print(outputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print("epoch: ",str(epoch))

    test_outputs = model(x_test.float())
    _, test_predicted = torch.max(test_outputs, 1)
    test_accuracy = (test_predicted == y_test).sum().item() / len(y_test)

    train_outputs = model(x_train.float())
    _, train_predicted = torch.max(train_outputs, 1)
    train_accuracy = (train_predicted ==
                        y_train).sum().item() / len(y_train)
    print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}, Test Accuracy: {test_accuracy * 100:.2f}%, Train Accuracy: {train_accuracy * 100:.2f}%')

    torch.save(model.state_dict(), "model/model1.pth")
    return model


In [14]:
def test():
    print()
    

In [15]:
import os
### The [0] gets the last frame, [5] would get 5th to last, etc.
path = "./avis_strike"
files = os.listdir(path)

strikes = []
for file in files:
    strikes.append(extract_frames(path+"/"+file, [0]))

path = "./avis_ball"
files = os.listdir(path)

balls = []
for file in files:
    balls.append(extract_frames(path+"/"+file, [0]))


In [9]:
b_idx = int(len(balls)*.75)
s_idx = int(len(strikes)*.75)

print(b_idx)
print(s_idx)

x_train = balls[:b_idx] + strikes[:s_idx]
x_train = torch.tensor(x_train)
x_train = torch.squeeze(x_train)

x_test = balls[b_idx:] + strikes[s_idx:]
x_test = torch.tensor(x_test)
x_test = torch.squeeze(x_test)

y_train = torch.tensor(np.concatenate((np.zeros( b_idx), np.ones(s_idx))))
y_test = torch.tensor(np.concatenate((np.zeros(len(balls)-b_idx), np.ones(len(strikes)-s_idx))))

print(x_test.shape)
print(x_train.shape)

print(y_test.shape)
print(y_train.shape)

11
26


/var/folders/nb/7_x2mx414t7fk4_n041gwvyc0000gn/T/ipykernel_49605/3893506297.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1711403226260/work/torch/csrc/utils/tensor_new.cpp:278.)
  x_train = torch.tensor(x_train)


torch.Size([13, 720, 1280, 3])
torch.Size([37, 720, 1280, 3])
torch.Size([13])
torch.Size([37])


In [22]:
train(x_train, x_test, y_train, y_test)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (10x3772 and 96x128)

**Start of Pitch-Type Classification**

In [14]:
import csv

filenames = []
with open('./pitch_types/curves.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)

    for row in reader:
        filenames.append(row['Title'])

print(filenames)

path = "./0strike_avis"
files = os.listdir(path)

curves = []
for file in files:
    if file in filenames:
        curves.append(extract_frames(path+"/"+file, [0]))

# All pitch types can be implemented the same as above, 
# path should include both 0strike_avis and 0ball_avis
sliders = []
fasts = []


curve_idx = int(len(curves)*.75)
slider_idx = int(len(sliders)*.75)
fast_idx = int(len(fasts)*.75)

x_train = curves[:curve_idx] + sliders[:slider_idx]+fasts[:fast_idx]
x_train = torch.tensor(x_train)
x_train = torch.squeeze(x_train)

x_test = curves[curve_idx:] + sliders[slider_idx:]+fasts[fast_idx:]
x_test = torch.tensor(x_test)
x_test = torch.squeeze(x_test)

y_train = torch.tensor(np.concatenate((np.zeros( curve_idx), np.ones(slider_idx), 2*np.ones(fast_idx))))
y_test = torch.tensor(np.concatenate((np.zeros(len(curves)-curve_idx), np.ones(len(sliders)-slider_idx), 2*np.ones(len(fasts)-fast_idx))))



['3263.1624651334137.avi', '607.7366191253092.avi', '5888.617335929688.avi', '3517.911483333333.avi', '855.7366191253092.avi', '11428.69556865428.avi', '2132.546333333333.avi', '4436.87213830851.avi', '1356.7394060727393.avi', '10042.162465133413.avi', '4170.6955686542815.avi', '8043.317851692016.avi', '3933.6677.avi', '1634.7366191253095.avi', '11995.73940607274.avi', '2104.546333333333.avi', '11413.69556865428.avi', '3342.67330666807.avi', '4274.546333333334.avi', '7907.009788958476.avi', '8439.587890111046.avi', '571.67330666807.avi', '12610.317851692016.avi', '5882.7366191253095.avi', '12811.739406072738.avi', '9948.162465133413.avi', '7833.009788958476.avi', '3952.6676999999995.avi', '7746.73661912531.avi', '5645.831219409505.avi', '12185.317851692018.avi', '4427.158933333333.avi', '1675.6733066680704.avi', '4529.546333333333.avi', '8743.321983333333.avi', '2594.6676999999995.avi', '4191.009788958476.avi', '5388.546333333334.avi', '5429.546333333334.avi', '602.8312194095048.avi', 